In [1]:
import numpy as np

# Input node matrix, where each row represents the [x,y] coordinates of that node
NODE = np.array([
    [0,0], 
    [0,2], 
    [0,4],
    [3,0],
    [3,2],
    [3,4],
    [6,2],
    [6,4],
    [9,2],
    [9,4],
    [12,0],
    [12,2],
    [12,4],
    [15,0],
    [15,2],
    [15,4] 
    ])
tnnd = len(NODE) # total number of nodes

# Input element matrix that establishes element-node connectivity
# Each row represents an element: [bottom left, bottom right, top right, top left] 
ELEM = np.array([
    [1,4,5,2], 
    [2,5,6,3], 
    [5,7,8,6],
    [7,9,10,8],
    [9,12,13,10],
    [12,15,16,13],
    [11,14,15,12]  
    ])
tnel = len(ELEM) # total number of elements

# Young's Modulus of each element
E = np.array([
    10e6,
    10e6,
    10e6,
    10e6,
    10e6,
    10e6,
    10e6
    ])

# Poisson's ratio of each element
Nu = np.array([
    .3,
    .3,
    .3,
    .3,
    .3,
    .3,
    .3
    ])

# Thickness of 2D element
t = .1

In [2]:
import input # import input module with NODE, ELEM matrices and other parameters
import sympy as sp
import numpy as np

def local_stiffness(elem_num): # for a given element number, will return LK as a matrix
    E = input.E[elem_num-1]
    Nu = input.Nu[elem_num-1]
    element = input.ELEM[elem_num-1] # returns the proper row of the ELEM matrix, which contains the associated nodes
    n1 = element[0] # bottom left node
    n2 = element[1] # bottom right node
    n4 = element[3] # top left node
    a = input.NODE[n2][0] - input.NODE[n1][0] # subtract the x coordinate of n1 from the x coordinate of n2 to get x length of element
    b = input.NODE[n4][1] - input.NODE[n1][1] # subtract the y coordinate of n1 from the y coordinate of n4 to get y length of element
    x = sp.Symbol('x') # define x symbolically
    y = sp.Symbol('y') # define y symbolically
    f1 = (1-(x/a))*(1-(y/b)) # first shape function for 8 DOF rectangular plane stress/plane strain finite element
    f2 = (x/a)*(1-(y/b)) # second shape function for 8 DOF rectangular plane stress/plane strain finite element
    f3 = x*y/a/b # third shape function for 8 DOF rectangular plane stress/plane strain finite element
    f4 = (1-(x/a))*y/b # fourth shape function for 8 DOF rectangular plane stress/plane strain finite element
    shape_function_list = [f1, f2, f3, f4] # create a list so the shape functions can be indexed
    
    def get_shape_functions(i,j): # function used within each of the 3 equations to determine fi and fj and their partial derivatives
        fi = shape_function_list[i-1] # pulls proper shape function for fi
        fj = shape_function_list[j-1] # pulls proper shape function for fj
        dfidx = sp.diff(fi,x) # get partial derivative dfi/dx
        dfjdx = sp.diff(fj,x) # get partial derivative dfj/dx
        dfidy = sp.diff(fi,y) # get partial derivative dfi/dy
        dfjdy = sp.diff(fj,x) # get partial derivative dfj/dy
        return dfidx, dfjdx, dfidy, dfjdy
    
    def double_integral(integrand): # function used to symbolically integrate from 0 to a WRT x, then from 0 to b WRT y
        I1 = sp.integrate(integrand,(x, 0, a)) #integrate with respect to x
        solved_integral = sp.integrate(I1, (y, 0, b)) #integrate with respect to y
        return solved_integral

    kuu = np.zeros((4,4)) # intialize kuu quadrant matrix
    kuv = np.zeros((4,4)) # intialize kuv quadrant matrix
    kvv = np.zeros((4,4)) # intialize kvv quadrant matrix
    multiplier = input.t * E/(1-Nu**2) # calculate constant that is multiplied by each stiffness matrix value
    for i in range(1,5): # iterates through each relative index location in the four quadrant matrices of the local stiffness matrix  
        for j in range(1,5):
            dfidx, dfjdx, dfidy, dfjdy = get_shape_functions(i,j) # call function to determine what fi, fj, and their partial derivatives are
            kuu[i-1,j-1] = multiplier * double_integral((dfidx*dfjdx) + (((1-Nu)/2)*dfidy*dfjdy)) # set the given value within the coefficient matrix for kuu
            kuv[i-1,j-1] = multiplier * double_integral((dfidy*dfjdy) + (((1-Nu)/2)*dfidx*dfjdx)) # set the given value within the coefficient matrix for kuv
            kvv[i-1,j-1] = multiplier * double_integral((Nu*dfidx*dfjdy) + (((1-Nu)/2)*dfidy*dfjdx)) # set the given value within the coefficient matrix for kvv
    LK = np.block([[kuu, kuv], [kuv, kvv]]) # Use kuu, kuv, and kvv to form full local stiffness matrix by concatenates quadrant matrices
    return LK

In [3]:
import input as ip
import local_stiffness as lk
import numpy as np

def global_stiffness(): # defines a function to be called to calculate the global stiffness matrix
    GK = np.zeros((2 * ip.tnnd, 2 * ip.tnnd)) # initialize global stiffness matrix
    for k in range(1,ip.tnel+1): # iterates from k = 1:tnel
        LK = lk.local_stiffness(k)  # precompute local stiffness matrix by calling the local_stiffness function within the local_stiffness module and determining LK at element #k
        element = ip.ELEM[k-1] # predefines the element row
        for i in range(1,9): # iterates from i = 1:8 for 2D
            if i <= 4: # within upper half or Fx portion of the local stiffness matrix
                ii = element[i-1] # sets the GK row index ii to be the node number of the given LK row 
            else: # within the bottom half or Fy portion of the local stiffness matrix
                i -= 4 # subtracts 4 from i so now Fy1 through Fy4 can be examined
                ii = element[i-1] + ip.tnnd # adds the total number of nodes to account for the shift to the bottom half of GK
            for j in range(1,9): # iterates from j = 1:8 for 2D
                if j <= 4: # within the left half or u portion of the local stiffness matrix
                    jj = element[j-1] # sets the GK column index jj to be the node number of the given LK column 
                else: # within the right half or v portion of the local stiffness matrix
                    j -= 4 # subtracts 4 from j so now v1 through v4 can be examined
                    jj = element[j-1] + ip.tnnd # adds the total number of elements to account for the shift to the right half of GK
                GK[ii-1,jj-1] += LK[i-1,j-1] # adds LK(i,j) to current value in GK
    return GK

In [4]:
import global_stiffness as gs
import numpy as np
import pandas as pd

GK = gs.global_stiffness() # calls the global_stiffness function within the global_stiffness module to obtain the global stiffness matrix, GK
np.set_printoptions(threshold=np.inf)
gk_visualization = pd.DataFrame(GK)
gk_visualization

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,340354.090354,218253.968254,0.000000,-340354.090354,-218253.968254,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,25946.275946,-244200.244200,-148046.398046,-25946.275946,244200.244200,148046.398046,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,-340354.090354,-584554.334554,0.000000,340354.090354,584554.334554,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,-148046.398046,-25946.275946,0.000000,148046.398046,25946.275946,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,-218253.968254,244200.244200,340354.090354,218253.968254,-392246.642247,-366300.366300,148046.398046,25946.275946,0.000000,0.000000,...,148046.398046,25946.275946,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.000000,148046.398046,392246.642247,0.000000,-366300.366300,-732600.732601,218253.968254,340354.090354,0.000000,0.000000,...,218253.968254,340354.090354,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.000000,340354.090354,218253.968254,-488400.488400,-244200.244200,148046.398046,25946.275946,...,-488400.488400,-244200.244200,148046.398046,25946.275946,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.000000,0.000000,0.000000,0.000000,25946.275946,148046.398046,-244200.244200,-488400.488400,218253.968254,340354.090354,...,-244200.244200,-488400.488400,218253.968254,340354.090354,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,340354.090354,218253.968254,-732600.732601,-366300.366300,...,340354.090354,218253.968254,-732600.732601,-366300.366300,0.000000,392246.642247,148046.398046,0.000000,0.000000,0.000000
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,25946.275946,148046.398046,-366300.366300,-732600.732601,...,25946.275946,148046.398046,-366300.366300,-732600.732601,0.000000,340354.090354,584554.334554,0.000000,0.000000,0.000000
